In [1]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import io
from skimage.segmentation import flood_fill
import cv2
import startn_utils as utils

In [2]:
suppress_plots = True

In [3]:
#major controlling parameters
tol_percent = 1
upper_limit = 120
area_percent = 3

crop = 41
crop_size = (np.floor(crop/2))
crop_size = int(crop_size)

In [4]:
# read data from tiff stack
full_scan = np.transpose(io.imread('data/example.tiff'),[1,2,0])
print(full_scan.shape)

(320, 320, 258)


In [5]:
centroid = np.array([137, 204, 155], dtype = np.uint16)

In [6]:
crop_scan_original = full_scan[centroid[0]-crop_size:centroid[0]+crop_size+1, centroid[1]-crop_size:centroid[1]+crop_size+1, centroid[2]-crop_size:centroid[2]+ crop_size+1]

if not suppress_plots:
    plt.imshow(crop_scan_original[:,20,:])

In [7]:
# saturate top 15% of values and normalize to 255
print(f"[INFO] Maximum value is {np.amax(crop_scan_original)}")
cutoff = (np.amax(crop_scan_original))*0.85
#normalizing the value
crop_scan_mask = crop_scan_original > cutoff #> 900
crop_scan_original[crop_scan_mask] = cutoff #900
scale_value = 255/(np.amax(crop_scan_original))
crop_scan = np.array(scale_value*crop_scan_original, dtype = np.uint16)

print(f"[INFO] New maximum value is {np.amax(crop_scan)}")
if not suppress_plots:
    plt.imshow(crop_scan[:,20,:])

[INFO] Maximum value is 986
[INFO] New maximum value is 255


In [8]:
#array to save segmentation
segmentation_y = np.zeros_like(crop_scan)
print(f"[INFO] Moving in the y direction")

segmentation_y[crop_size,crop_size,crop_size] = 1
#filling center of segmentation with 1 (meaning true)
print(f"[INFO] Acceptable area change for cross-section is {area_percent}") #150%

[INFO] Moving in the y direction
[INFO] Acceptable area change for cross-section is 3


In [9]:
#pull out the initial array fo r a given z
arr1_og = crop_scan[:, 20, :]

arr1_blurred = cv2.GaussianBlur(arr1_og, (3,3), 0)
#calculate scale value to scale to uint8 scale
dat = np.array(arr1_blurred, dtype = np.uint8)

if not suppress_plots:
    plt.figure(figsize=(10,5))
    plt.subplot(121)
    plt.imshow(arr1_og)
    plt.colorbar()
    plt.subplot(122)
    plt.imshow(dat)
    plt.colorbar()

In [10]:
#blur to remove noise
arr1_binary_all = cv2.adaptiveThreshold(dat, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 7, 0)
centerPix = dat[crop_size,crop_size]
th = (np.percentile(dat.flatten(),95) - centerPix)/3 + centerPix
_, arr1_binary_thresh = cv2.threshold(dat, th, 255, cv2.THRESH_BINARY)

if not suppress_plots:
    plt.figure(figsize=(10,5))
    plt.subplot(121)
    plt.imshow(arr1_binary_thresh)
    plt.colorbar()
    plt.subplot(122)
    plt.imshow(arr1_binary_all)
    plt.colorbar()

In [11]:
arr1_binary_all = arr1_binary_all | arr1_binary_thresh

init_guess = np.zeros_like(arr1_binary_all)
init_guess[17:23,17:23] = 1

arr1_binary = utils.remove_background(arr1_binary_all, init_guess)
arr1_binary = arr1_binary.astype(np.uint8)

if not suppress_plots:
    plt.imshow(arr1_binary)
    plt.axis('off')

In [12]:
arr1 = utils.hull_and_kmeans(arr1_binary)

if arr1[crop_size, crop_size] > 200:
    arr1[crop_size, crop_size] = 14

arr1[arr1 == 255] = 200

if not suppress_plots:
    plt.imshow(arr1)
    plt.axis('off')

In [13]:
#flood fill from teh 2D centroid
arr1_flooded = flood_fill(arr1, (crop_size, crop_size), 1, connectivity = 1, tolerance = upper_limit - np.ceil(arr1[crop_size, crop_size]))
arr1_flooded = arr1_flooded == 1

if not suppress_plots:
    plt.imshow(arr1_flooded)
    plt.colorbar()

In [14]:
arr1_old = arr1_flooded.copy();
tmp = np.zeros((crop,crop));
for i in range(arr1_flooded.shape[0]):
    for j in range(arr1_flooded.shape[1]):
        if arr1_old[i,j] == True:
            if i > 0 and arr1[i-1,j] == 254 and dat[i-1,j] < 130:
                tmp[i-1,j] = tmp[i-1,j] + 1;
            if j > 0 and arr1[i,j-1] == 254 and dat[i,j-1] < 130:
                tmp[i,j-1] = tmp[i,j-1] + 1;
            if i < arr1_flooded.shape[0]-1 and arr1[i+1,j] == 254 and dat[i+1,j] < 130:
                tmp[i+1,j] = tmp[i+1,j] + 1;
            if j < arr1_flooded.shape[1]-1 and arr1[i,j+1] == 254 and dat[i,j+1] < 130:
                tmp[i,j+1] = tmp[i,j+1] + 1;
            if i > 0 and j > 0 and arr1[i-1,j-1] == 254 and dat[i-1,j-1] < 130:
                tmp[i-1,j-1] = tmp[i-1,j-1] + 1;
            if i > 0 and j < arr1_flooded.shape[1]-1 and arr1[i-1,j+1] == 254 and dat[i-1,j+1] < 130:
                tmp[i-1,j+1] = tmp[i-1,j+1] + 1;
            if i < arr1_flooded.shape[0]-1 and j > 0 and arr1[i+1,j-1] == 254 and dat[i+1,j-1] < 130:
                tmp[i+1,j-1] = tmp[i+1,j-1] + 1;
            if  i < arr1_flooded.shape[0]-1 and j < arr1_flooded.shape[1]-1 and arr1[i+1,j+1] == 254 and dat[i+1,j+1] < 130:
                tmp[i+1,j+1] = tmp[i+1,j+1] + 1;
arr1_flooded = arr1_flooded | (tmp > 3);

if not suppress_plots:
    plt.imshow(arr1_flooded)
    plt.colorbar()

In [15]:
arr3 = arr1_binary
arr3[arr1_flooded] = 254

if not suppress_plots:
    plt.imshow(arr3)
    plt.colorbar()

In [16]:
arr1[arr1_flooded] = 255

if not suppress_plots:
    plt.imshow(arr1)
    plt.axis('off')

In [17]:
arr1_dilated = cv2.dilate(np.asarray(arr1_flooded*255, dtype="uint8"), np.ones((2,2)))
print(f"[INFO] Initialization area is {sum(sum(arr1_flooded))}")
out, flag = utils.loop(arr3, sum(sum(arr1_flooded)), arr1_dilated == 255, arr1_flooded)
arr1_flooded = out | arr1_flooded

if not suppress_plots:
    plt.imshow(arr1_flooded)
    plt.colorbar()

[INFO] Initialization area is 16
Calculated area for next loop is 0


In [18]:
#calculate area, store area as baseline for later comparison
area_at_centroid = np.sum(np.sum(arr1_flooded))
major_axis_a = np.sum(arr1_flooded,axis = 0)[crop_size]
major_axis_b = np.sum(arr1_flooded,axis=1)[crop_size]
ellipse_area = 3.1415926 * major_axis_a * major_axis_b

segmentation_y[:, crop_size, :] = arr1_flooded

In [19]:
utils.press_forward_y(arr1_flooded, crop_size, area_at_centroid,crop_scan,segmentation_y)

Calculated area for next loop is 17
[FORWARD] Calculated area for next forwards step is 17
Previous area was 16
Calculated area for next loop is 18
[FORWARD] Calculated area for next forwards step is 18
Previous area was 17
Calculated area for next loop is 14
[FORWARD] Calculated area for next forwards step is 14
Previous area was 18
Calculated area for next loop is 18
[FORWARD] Calculated area for next forwards step is 18
Previous area was 14
Calculated area for next loop is 18
[FORWARD] Calculated area for next forwards step is 18
Previous area was 18
Calculated area for next loop is 0
Original area size is 0
Calculated area for next loop is 22
Calculated area for next loop is 4
Filled in the gaps with area 26 and allowable change is 54 at 26
[FORWARD] Calculated area for next forwards step is 26
Previous area was 18
Calculated area for next loop is 0
Original area size is 0
Calculated area for next loop is 24
Calculated area for next loop is 4
Filled in the gaps with area 28 and all

In [20]:
utils.press_backward_y(arr1_flooded, crop_size, area_at_centroid,crop_scan,segmentation_y)

Calculated area for next loop is 18
[BACK] Calculated area for next backwards step is 18
Previous area was 16
Calculated area for next loop is 21
[BACK] Calculated area for next backwards step is 21
Previous area was 18
Calculated area for next loop is 26
[BACK] Calculated area for next backwards step is 26
Previous area was 21
Calculated area for next loop is 23
Original area size is 23
Calculated area for next loop is 23
Calculated area for next loop is 11
Filled in the gaps with area 34 and allowable change is 78 at 16
[BACK] Calculated area for next backwards step is 34
Previous area was 26
Calculated area for next loop is 0
Original area size is 0
Calculated area for next loop is 26
Calculated area for next loop is 10
Filled in the gaps with area 36 and allowable change is 102 at 15
[BACK] Calculated area for next backwards step is 36
Previous area was 34
Calculated area for next loop is 0
Original area size is 0
Calculated area for next loop is 25
Calculated area for next loop is

/Users/michael/anaconda3/envs/tn-seg/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/michael/anaconda3/envs/tn-seg/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/michael/anaconda3/envs/tn-seg/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/michael/anaconda3/envs/tn-seg/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/michael/anaconda3/envs/tn-seg/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [21]:
if not suppress_plots:
    fig = plt.figure(figsize=(10,10))
    _, axes = fig.subplots(2,2)
    s0 = crop_size
    c0 = crop_size
    a0 = crop_size


    coord = np.array([s0,c0,a0])
    gridlinestyles = 'dashed'

    plt.subplot(221)
    im_s = plt.imshow(crop_scan[:,:,s0], cmap = plt.cm.gray)#, vmin=lb, vmax=ub)
    seg_s = plt.imshow(segmentation_y[:,:,s0], alpha = 0.5,vmin = 0, vmax = 1)
    al_s = plt.hlines(a0,xmin=0,xmax=crop_scan.shape[1]-1,linestyles=gridlinestyles)
    cl_s = plt.vlines(c0,ymin=0,ymax=crop_scan.shape[0]-1,linestyles=gridlinestyles)
    plt.subplot(222)
    im_c = plt.imshow(crop_scan[:,c0,:], cmap = plt.cm.gray)#, vmin=lb, vmax=ub)
    seg_c = plt.imshow(segmentation_y[:,c0,:], alpha = 0.5,vmin = 0, vmax = 1)
    al_c = plt.hlines(a0,xmin=0,xmax=crop_scan.shape[2]-1,linestyles=gridlinestyles)
    sl_c = plt.vlines(s0,ymin=0,ymax=crop_scan.shape[0]-1,linestyles=gridlinestyles)
    plt.subplot(223)
    im_a = plt.imshow(crop_scan[a0,:,:], cmap = plt.cm.gray)#, vmin=lb, vmax=ub)
    seg_a = plt.imshow(segmentation_y[a0,:,:], alpha = 0.5,vmin = 0, vmax = 1)
    cl_a = plt.hlines(c0,xmin=0,xmax=crop_scan.shape[2]-1,linestyles=gridlinestyles)
    sl_a = plt.vlines(s0,ymin=0,ymax=crop_scan.shape[1]-1,linestyles=gridlinestyles)

In [22]:
segmentation = np.array(np.zeros_like(full_scan), dtype=np.uint8)

segmentation[centroid[0]-crop_size:centroid[0]+crop_size+1, centroid[1]-crop_size:centroid[1]+crop_size+1, centroid[2]-crop_size:centroid[2]+ crop_size+1] = np.array(segmentation_y.copy(),dtype=np.uint8)


In [23]:
if not suppress_plots:
    fig = plt.figure(figsize=(10,10))
    _, axes = fig.subplots(2,2)
    s0 = centroid[2]
    c0 = centroid[1]
    a0 = centroid[0]


    coord = np.array([s0,c0,a0])
    gridlinestyles = 'dashed'

    plt.subplot(221)
    im_s = plt.imshow(full_scan[:,:,s0], cmap = plt.cm.gray)#, vmin=lb, vmax=ub)
    seg_s = plt.imshow(segmentation[:,:,s0], alpha = 0.5,vmin = 0, vmax = 1)
    al_s = plt.hlines(a0,xmin=0,xmax=full_scan.shape[1]-1,linestyles=gridlinestyles)
    cl_s = plt.vlines(c0,ymin=0,ymax=full_scan.shape[0]-1,linestyles=gridlinestyles)
    plt.subplot(222)
    im_c = plt.imshow(full_scan[:,c0,:], cmap = plt.cm.gray)#, vmin=lb, vmax=ub)
    seg_c = plt.imshow(segmentation[:,c0,:], alpha = 0.5,vmin = 0, vmax = 1)
    al_c = plt.hlines(a0,xmin=0,xmax=full_scan.shape[2]-1,linestyles=gridlinestyles)
    sl_c = plt.vlines(s0,ymin=0,ymax=full_scan.shape[0]-1,linestyles=gridlinestyles)
    plt.subplot(223)
    im_a = plt.imshow(full_scan[a0,:,:], cmap = plt.cm.gray)#, vmin=lb, vmax=ub)
    seg_a = plt.imshow(segmentation[a0,:,:], alpha = 0.5,vmin = 0, vmax = 1)
    cl_a = plt.hlines(c0,xmin=0,xmax=full_scan.shape[2]-1,linestyles=gridlinestyles)
    sl_a = plt.vlines(s0,ymin=0,ymax=full_scan.shape[1]-1,linestyles=gridlinestyles)

In [24]:
#save the segmentation as tiff
io.imsave('data/segmentation.tiff', segmentation.transpose([2,0,1]))

/var/folders/kg/kdvbybgj03g64srr848x9b500000gn/T/ipykernel_36552/3356138330.py:2: UserWarning: data/segmentation.tiff is a low contrast image
  io.imsave('data/segmentation.tiff', segmentation.transpose([2,0,1]))
